<a href="https://colab.research.google.com/github/maykon/intent-classifier/blob/master/Intent_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


In [0]:
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = 'utf-8', names = ["Sentence", "Intent"])
  print(df.head())
  intent = df["Intent"]
  unique_intent = list(set(intent))
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)
  


In [0]:
intent, unique_intent, sentences = load_dataset("Dataset_pt.csv")

                              Sentence       Intent
0                        Frete grátis?  FreteGratis
1                   Quanto tá o frete?   DadosFrete
2  qual o valor do frete para maringá?   DadosFrete
3       quanto ta o frete pra maringá?   DadosFrete
4        quanto ta o frte para maringá   DadosFrete


In [0]:
print(sentences[:5])

['Frete grátis?', 'Quanto tá o frete?', 'qual o valor do frete para maringá?', 'quanto ta o frete pra maringá?', 'quanto ta o frte para maringá']


In [0]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('rslp')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [0]:
#define stemmer
stemmer = RSLPStemmer()

In [0]:
def cleaning(sentences):
  words = []
  for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean, language='portuguese')
    #stemming
    words.append([stemmer.stem(i.lower()) for i in w])
    
  return words  

In [0]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
  


32
[['fret', 'gr', 'ti'], ['quant', 't', 'o', 'fret']]


In [0]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [0]:
def max_length(words):
  return(len(max(words, key = len)))
  

In [0]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 69 and Maximum length = 14


In [0]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [0]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [0]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [0]:
padded_doc = padding_doc(encoded_doc, max_length)

In [0]:
padded_doc[:5]

array([[ 2,  6,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 4, 20,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [21,  1, 31, 32,  2,  5,  8,  0,  0,  0,  0,  0,  0,  0],
       [ 4, 10,  1,  2, 11,  8,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 4, 10,  1, 33,  5,  8,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int32)

In [0]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (32, 14)


In [0]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')


In [0]:
output_tokenizer.word_index

{'dadosfrete': 2, 'dadosnotafiscal': 1, 'fretegratis': 3}

In [0]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [0]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [0]:
encoded_output.shape

(32, 1)

In [0]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [0]:
output_one_hot = one_hot(encoded_output)

In [0]:
output_one_hot.shape

(32, 3)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)


In [0]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (25, 14) and train_Y = (25, 3)
Shape of val_X = (7, 14) and val_Y = (7, 3)


In [0]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(128)))
#   model.add(LSTM(128))
  model.add(Dense(20, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(3, activation = "softmax"))
  
  return model

In [0]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 14, 128)           8832      
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 256)               263168    
_________________________________________________________________
dense_19 (Dense)             (None, 20)                5140      
_________________________________________________________________
dropout_10 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 3)                 63        
Total params: 277,203
Trainable params: 268,371
Non-trainable params: 8,832
_________________________________________________________________


In [0]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 50, batch_size = 10, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Train on 25 samples, validate on 7 samples
Epoch 1/50
25/25 [==============================] - 5s 185ms/step - loss: 1.1026 - acc: 0.2400 - val_loss: 1.0968 - val_acc: 0.1429

Epoch 00001: val_loss improved from inf to 1.09680, saving model to model.h5
Epoch 2/50
25/25 [==============================] - 0s 4ms/step - loss: 1.0940 - acc: 0.2800 - val_loss: 1.0885 - val_acc: 0.5714

Epoch 00002: val_loss improved from 1.09680 to 1.08850, saving model to model.h5
Epoch 3/50
25/25 [==============================] - 0s 4ms/step - loss: 1.0743 - acc: 0.4800 - val_loss: 1.0840 - val_acc: 0.5714

Epoch 00003: val_loss improved from 1.08850 to 1.08401, saving model to model.h5
Epoch 4/50
25/25 [==============================] - 0s 4ms/step - loss: 1.0731 - acc: 0.3600 - val_loss: 1.0782 - val_acc: 0.4286

Epoch 00004: val_loss improved from 1.08401 to 1.07817, saving model to model.h5
Epoch 5/50
25/25 [==============================] - 0s 5ms/step - loss: 1.0412 - acc: 0.5200 - val_loss: 1.0680

In [0]:
 model = load_model("model.h5")

In [0]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean, language='portuguese')
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred


  

In [0]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [0]:
text = "o aviso fala frete gratis."
pred = predictions(text)
get_final_output(pred, unique_intent)

['o', 'aviso', 'fala', 'frete', 'gratis']
FreteGratis has confidence = 0.54609555
DadosFrete has confidence = 0.44028553
DadosNotaFiscal has confidence = 0.013618923
